In [1]:
!pip install schedule

In [2]:
%run ../spark_session.ipynb

In [3]:
import time
import requests
import schedule
from pyspark.sql.functions import to_timestamp

In [4]:
def job_postgre_position():
    print("Criando sessão spark e carregando dados /trusted/position")
    spark = get_or_create_session_spark("postgre_trusted_position")
    
    df = spark.read.format("delta").load("s3a://trusted/position")
    df = df.withColumn("data_hora", to_timestamp("data_hora"))
    
    jdbc_url = "jdbc:postgresql://db:5432/spacedb"
    properties = {
        "user": "spacedb",
        "password": "spacedb",
        "driver": "org.postgresql.Driver"
    }

    # Escrever dados no PostgreSQL
    df.write.jdbc(url=jdbc_url, table="spacedb.position", mode="append", properties=properties)
    spark.stop()

    print("Dados escritos com sucesso na tabela PostgreSQL!")

In [ ]:
schedule.every(5).minutes.do(job_postgre_position)

while True:
    schedule.run_pending()
    time.sleep(1)

Criando sessão spark e carregando dados /trusted/position
Obtendo sessão spark postgre_trusted_position
Sessão Spark obtida com sucesso! postgre_trusted_position
Dados escritos com sucesso na tabela PostgreSQL!
